Ici on récupère les données via PISTE l'API permettant d'avoir accès aux données de Légifrance, le code de connexion OAuth est donné sur le Gitlab de Piste à cette [adresse](https://gitlab.com/piste_lab/oauth_connectors/-/blob/master/Python/Oauth2ClientCredentialsTest.py?ref_type=heads). Il est tout de même nécessaire d'installer les modules du code suivant.

In [ ]:
%pip install python-dotenv
%pip install requests-oauthlib
%

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Pour pour envoyer des requêtes à l'API nous devons d'abord obtenir un token, après s'être inscrit sur le [site de PISTE](https://piste.gouv.fr/). Ce token est obtenu en envoyant une requête contenant mon identifiant client et mon code de client au site d'autorisation. Nous avons ensuite une autorisation d'une heure avec ce token pour exploiter l'API de Légifrance. 

In [2]:
TOKEN_URL = "https://oauth.piste.gouv.fr/api/oauth/token"

import os
from dotenv import load_dotenv
import requests
from requests_oauthlib import OAuth2Session

# Cette partie sert à récupérer les ID de connexions qui sont dans un fichier .env que l'on doit créer à chaque fois que l'on copie le repository github, cela permet de les conserver secret
load_dotenv()
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

# Requette d'un token pour avoir accès à l'API 
res = requests.post(
  TOKEN_URL,
  data={
    "grant_type": "client_credentials",
    "client_id": client_id,
    "client_secret": client_secret,
    "scope": "openid"
  }
)

# Récupération du token 
token = res.json()
client = OAuth2Session(client_id, token=token)

Maintenant en utilisant le lien d'exploitation de l'API auquel on rajoute l'endpoints qui permet d'accéder à ce que l'on veut faire nous récupérons les données . La liste des endpoints pour l'API de Légifrance est disponible [ici](https://piste.gouv.fr/index.php?option=com_apiportal&view=apitester&usage=api&apitab=tests&apiName=L%C3%A9gifrance&apiId=7e5a0e1d-ffcc-40be-a405-a1a5c1afe950&managerId=3&type=rest&apiVersion=2.4.2&Itemid=179&swaggerVersion=2.0&lang=fr).

In [3]:
API_HOST = "https://api.piste.gouv.fr/dila/legifrance/lf-engine-app"

In [25]:
import requests
import pandas as pd
import math

code_api = {
  "recherche": {
    "filtres": [
      {
        "dates": {
          "start": "1996-01-01",
          "end": "2022-08-31"
        },
        "facette": "DATE_SIGNATURE"
      }
    ],
    "sort": "SIGNATURE_DATE_DESC",
    "fromAdvancedRecherche": False,
    "champs": [
      {
        "criteres": [
          {
            "valeur": "délinquance",
            "operateur": "OU",
            "criteres": [
              {
                "valeur": "crime",
                "operateur": "OU",
                "typeRecherche": "UN_DES_MOTS"
              },
              {
                "valeur": "délit",
                "operateur": "OU",
                "typeRecherche": "UN_DES_MOTS"
              }
            ],
            "typeRecherche": "UN_DES_MOTS"
          }
        ],
        "operateur": "ET",
        "typeChamp": "ALL"
      }
    ],
    "pageSize": 100,
    "pageNumber": 1,
    "operateur": "ET",
    "typePagination": "DEFAUT"
  },
  "fond": "LODA_DATE"
  }



def collect_all_results(client, api_host, code):
    # Récupérer le total de résultats et calculer le nombre de pages
    response = client.post(api_host, json=code).json()
    total_results = response.get("totalResultNumber", 0)
    page_size = code["recherche"]["pageSize"]
    total_pages = math.ceil(total_results / page_size)

    print(f"Total de résultats : {total_results}")
    print(f"Nombre de pages à récupérer : {total_pages}")

    # Liste pour stocker tous les résultats
    all_results = []
    for page_number in range(1, total_pages + 1):
        print(f"Récupération de la page {page_number}/{total_pages}...")
        code["recherche"]["pageNumber"] = page_number
        response = client.post(api_host, json=code).json()
        all_results.extend(response.get("results", []))

    return all_results


In [29]:
all_results = collect_all_results(client, API_HOST+"/search", code_api)
def results_to_dataframe(results):
    data = []
    for result in results:
        title = result.get("titles", [{}])[0].get("title") if result.get("titles") else None
        date = result.get("date")
        data.append({"Titre": title, "Date": date})
    return pd.DataFrame(data)

df = results_to_dataframe(all_results)

Total de résultats : 344
Nombre de pages à récupérer : 4
Récupération de la page 1/4...
Récupération de la page 2/4...
Récupération de la page 3/4...
Récupération de la page 4/4...


In [30]:
df

,Titre,Date
0,Arrêté du 20 janvier 2022 fixant la liste des ...,2022-01-01T00:00:00.000+0000
1,Arrêté du 20 janvier 2022 fixant la liste des ...,2022-01-01T00:00:00.000+0000
2,Arrêté du 15 décembre 2021 fixant les règles d...,2022-01-01T00:00:00.000+0000
3,Arrêté du 1er juillet 2021 fixant la liste des...,2021-01-01T00:00:00.000+0000
4,Arrêté du 1er juillet 2021 fixant la liste des...,2021-01-01T00:00:00.000+0000
...,...,...
339,Loi n° 2002-1094 du 29 août 2002 d'orientation...,2002-08-30T00:00:00.000+0000
340,Loi n° 2002-1094 du 29 août 2002 d'orientation...,2002-08-30T00:00:00.000+0000
341,Loi n° 2002-1094 du 29 août 2002 d'orientation...,2002-08-30T00:00:00.000+0000
342,Loi n° 2002-1094 du 29 août 2002 d'orientation...,2002-08-30T00:00:00.000+0000
